# 分类数据

In [1]:
import numpy as np
import pandas as pd

## cat对象

### cat对象的属性
    在 pandas 中提供了 category 类型，使用户能够处理分类类型的变量，将一个普通序列转换成分类变量可以使用 astype 方法。

In [2]:
df = pd.read_csv('data/learn_pandas.csv',
     usecols = ['Grade', 'Name', 'Gender', 'Height', 'Weight'])
s = df.Grade.astype('category')

s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Junior', 'Senior', 'Sophomore']

    在一个分类类型的 Series 中定义了 cat 对象，它和上一章中介绍的 str 对象类似，定义了一些属性和方法来进行分类类别的操作。

In [3]:
s.cat

    对于一个具体的分类，有两个组成部分，其一为类别的本身，它以 Index 类型存储，其二为是否有序，它们都可以通过 cat 的属性被访问

In [4]:
s.cat.categories

Index(['Freshman', 'Junior', 'Senior', 'Sophomore'], dtype='object')

In [5]:
s.cat.ordered

False

    另外，每一个序列的类别会被赋予唯一的整数编号，它们的编号取决于 cat.categories 中的顺序，该属性可以通过 codes 访问

In [6]:
s.cat.codes.head()

0    0
1    0
2    2
3    3
4    3
dtype: int8

### 类别的增加、删除和修改

###### 增加
    对于类别的增加可以使用 add_categories

In [7]:
s = s.cat.add_categories('Graduate') # 增加一个毕业生类别
s.cat.categories

Index(['Freshman', 'Junior', 'Senior', 'Sophomore', 'Graduate'], dtype='object')

###### 删除
    若要删除某一个类别可以使用 remove_categories ，同时所有原来序列中的该类会被设置为缺失

In [8]:
s = s.cat.remove_categories('Freshman')
s.cat.categories

Index(['Junior', 'Senior', 'Sophomore', 'Graduate'], dtype='object')

In [9]:
s.head()

0          NaN
1          NaN
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Junior', 'Senior', 'Sophomore', 'Graduate']

###### 增加与删除
    可以使用 set_categories 直接设置序列的新类别，原来的类别中如果存在元素不属于新类别，那么会被设置为缺失。

In [10]:
s = s.cat.set_categories(['Sophomore','PhD']) # 新类别为大二学生和博士
s.cat.categories

Index(['Sophomore', 'PhD'], dtype='object')

In [11]:
s.head()

0          NaN
1          NaN
2          NaN
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (2, object): ['Sophomore', 'PhD']

###### 删除
    如果想要删除未出现在序列中的类别，可以使用 remove_unused_categories 来实现

In [12]:
s = s.cat.remove_unused_categories() # 移除了未出现的博士生类别

s.cat.categories

Index(['Sophomore'], dtype='object')

###### 修改
    修改的操作可以通过 rename_categories 方法完成，同时需要注意的是，这个方法会对原序列的对应值也进行相应修改

In [13]:
s = s.cat.rename_categories({'Sophomore':'本科二年级学生'})
s.head()

0        NaN
1        NaN
2        NaN
3    本科二年级学生
4    本科二年级学生
Name: Grade, dtype: category
Categories (1, object): ['本科二年级学生']

## 有序分类

### 序的建立
    有序类别和无序类别可以通过 as_unordered 和 reorder_categories 互相转化。
    后者传入的参数必须是由当前序列的无序类别构成的列表，不能够增加新的类别，也不能缺少原来的类别，并且必须指定参数 ordered=True ，否则方法无效。

In [14]:
s = df.Grade.astype('category')
print(s.head(),'\n')
s = s.cat.reorder_categories(['Freshman', 'Sophomore','Junior', 'Senior'],ordered=True)
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Junior', 'Senior', 'Sophomore'] 



0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman' < 'Sophomore' < 'Junior' < 'Senior']

In [15]:
s.cat.as_unordered().head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Sophomore', 'Junior', 'Senior']

### 排序和比较
    分类变量的排序：只需把列的类型修改为 category 后，再赋予相应的大小关系，就能正常地使用 sort_index 和 sort_values
    
    分类变量的比较操作分为两类:
        第一种是 == 或 != 关系的比较，比较的对象可以是标量或者同长度的 Series （或 list ）
        第二种是 >,>=,<,<= 四类大小关系的比较，比较的对象和第一种类似，但是所有参与比较的元素必须属于原序列的 categories ，同时要和原序列具有相同的索引

In [18]:
(df.Grade == 'Sophomore').head()

0    False
1    False
2    False
3     True
4     True
Name: Grade, dtype: bool

In [19]:
(df.Grade == ['PhD']*df.shape[0]).head()

0    False
1    False
2    False
3    False
4    False
Name: Grade, dtype: bool

In [20]:
(df.Grade <= 'Sophomore').head()

0    True
1    True
2    True
3    True
4    True
Name: Grade, dtype: bool

## 区间类别

### 利用cut和qcut进行区间构造
    区间序列往往是通过 cut 和 qcut 方法进行构造的。
    
    最重要的参数是 bins ，如果传入整数 n：

In [21]:
s = pd.Series([1,2])
pd.cut(s, bins=2)

0    (0.999, 1.5]
1      (1.5, 2.0]
dtype: category
Categories (2, interval[float64, right]): [(0.999, 1.5] < (1.5, 2.0]]

In [22]:
pd.cut(s, bins=2, right=False)

0      [1.0, 1.5)
1    [1.5, 2.001)
dtype: category
Categories (2, interval[float64, left]): [[1.0, 1.5) < [1.5, 2.001)]

    bins 的另一个常见用法是指定区间分割点的列表

In [23]:
pd.cut(s, bins=[-np.infty, 1.2, 1.8, 2.2, np.infty])

0    (-inf, 1.2]
1     (1.8, 2.2]
dtype: category
Categories (4, interval[float64, right]): [(-inf, 1.2] < (1.2, 1.8] < (1.8, 2.2] < (2.2, inf]]

    另外两个常用参数为 labels 和 retbins ，分别代表了区间的名字和是否返回分割点（默认不返回）

In [24]:
s = pd.Series([1,2])
res = pd.cut(s, bins=2, labels=['small', 'big'], retbins=True)
res[0]

0    small
1      big
dtype: category
Categories (2, object): ['small' < 'big']

    从用法上来说， qcut 和 cut 几乎没有差别，只是把 bins 参数变成的 q 参数， qcut 中的 q 是指 quantile 。这里的 q 为整数 n 时，指按照 n 等分位数把数据分箱，还可以传入浮点列表指代相应的分位数分割点。

In [27]:
s = df.Weight
print(pd.qcut(s, q=3).head(),'\n\n')
print(pd.qcut(s, q=[0,0.2,0.8,1]).head())

0    (33.999, 48.0]
1      (55.0, 89.0]
2      (55.0, 89.0]
3    (33.999, 48.0]
4      (55.0, 89.0]
Name: Weight, dtype: category
Categories (3, interval[float64, right]): [(33.999, 48.0] < (48.0, 55.0] < (55.0, 89.0]] 


0      (44.0, 69.4]
1      (69.4, 89.0]
2      (69.4, 89.0]
3    (33.999, 44.0]
4      (69.4, 89.0]
Name: Weight, dtype: category
Categories (3, interval[float64, right]): [(33.999, 44.0] < (44.0, 69.4] < (69.4, 89.0]]


### 一般区间的构造
    对于某一个具体的区间而言，其具备三个要素，即左端点、右端点和端点的开闭状态，其中开闭状态可以指定 right, left, both, neither 中的一类
    其属性包含了 mid, length, right, left, closed ，分别表示中点、长度、右端点、左端点和开闭状态。

In [28]:
my_interval = pd.Interval(0, 1, 'right')
my_interval

Interval(0, 1, closed='right')

In [30]:
print(my_interval.mid)
print(my_interval.length)
print(my_interval.right)
print(my_interval.left)
print(my_interval.closed)

0.5
1
1
0
right


    使用 in 可以判断元素是否属于区间

In [31]:
0.5 in my_interval

True

    使用 overlaps 可以判断两个区间是否有交集

In [33]:
my_interval_2 = pd.Interval(0.5, 1.5, 'left')
my_interval.overlaps(my_interval_2)

True